In [1]:
import trimesh

scene = trimesh.Scene()

In [2]:
import pyvirtualdisplay
import trimesh
import my_code.diffusion_training_sign_corr.data_loading as data_loading
import yaml
import json
from tqdm import tqdm
import torch
import numpy as np
import trimesh.scene
import trimesh.scene.lighting
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
import PIL.Image



dataset_name = 'FAUST_r_pair'

single_dataset, pair_dataset = data_loading.get_val_dataset(
    dataset_name, 'test', 128, preload=False, return_evecs=True, centering='bbox'
)


if dataset_name == 'SHREC19_r_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_1-2ev_64-128-128_remeshed_fixed/eval/epoch_99/SHREC19_r_pair-test/no_smoothing/2024-11-04_22-27-59/pairwise_results.json'
elif dataset_name == 'DT4D_intra_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_template_remeshed/eval/checkpoint_99.pt/DT4D_intra_pair-test/no_smoothing/2024-11-10_21-20-05/pairwise_results.json'
elif dataset_name == 'DT4D_inter_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_template_remeshed/eval/checkpoint_99.pt/DT4D_inter_pair-test/no_smoothing/2024-11-10_21-20-05/pairwise_results.json'
elif dataset_name == 'FAUST_r_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_1-2ev_64-128-128_remeshed_fixed/eval/epoch_99/FAUST_r_pair-test/no_smoothing/2024-11-04_22-27-59/pairwise_results.json'
elif dataset_name == 'SCAPE_r_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_1-2ev_64-128-128_remeshed_fixed/eval/epoch_99/SCAPE_r_pair-test/no_smoothing/2024-11-04_22-27-59/pairwise_results.json'
elif dataset_name == 'SMAL_nocat_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_SMAL_nocat_64_SMAL_isoRemesh_0.2_0.8_nocat_1-2ev_64k/eval/epoch_99/SMAL_nocat_pair-test/no_smoothing/2025-01-24_16-01-31/pairwise_results.json'
        
with open(file_name, 'r') as f:
    p2p_saved = json.load(f)


geo_err_list = torch.tensor([p2p_saved[i]['geo_err_median_pairzo'] for i in range(len(p2p_saved))])
idxs_geo_err = torch.argsort(geo_err_list, descending=True)

        

In [46]:
indx = 1

data_i = pair_dataset[indx]
p2p_i = p2p_saved[indx]
p2p_pairzo = torch.tensor(p2p_i['p2p_median_pairzo'])

print(p2p_i['geo_err_median_pairzo'])


In [47]:
import utils.texture_util as texture_util

uv1 = texture_util.generate_tex_coords(data_i['first']['verts'].numpy())

# uv2 = uv1[p2p_pairzo]

In [48]:
Cxy = torch.linalg.lstsq(
    data_i['second']['evecs'],
    data_i['first']['evecs'][p2p_pairzo],
).solution

# Cxy = torch.linalg.lstsq(
#     data_i['second']['evecs'][data_i['second']['corr']],
#     data_i['first']['evecs'][data_i['first']['corr']],
# ).solution

Pyx = data_i['second']['evecs'] @ Cxy @ data_i['first']['evecs_trans']

uv2 = Pyx @ uv1

In [49]:
# read /home/s94zalek_hpc/shape_matching/figures/texture.png with PIL
from PIL import Image

#########################################
# X -> Y
#########################################

scene.geometry.clear()


# texture_img = Image.open('/home/s94zalek_hpc/shape_matching/figures/texture.png')
texture_img = Image.open('/home/s94zalek_hpc/shape_matching/figures/Custom_texture_w_border.png')

# create material
material=trimesh.visual.material.SimpleMaterial(
        image=texture_img,
        diffuse=[255, 255, 255, 255],
    )

# add the first mesh
mesh1 = trimesh.Trimesh(
    vertices=data_i['first']['verts'].numpy(), 
    faces=data_i['first']['faces'].numpy()
    )

texture_visuals = trimesh.visual.texture.TextureVisuals(
    uv=uv1[:len(mesh1.vertices)],
    material=material
)

mesh1.visual = texture_visuals

scene.add_geometry(mesh1)


# correspondence we got from the functional map
mesh2_after = trimesh.Trimesh(
    vertices=data_i['second']['verts'].numpy() + np.array([1, 0, 0]), 
    faces=data_i['second']['faces'].numpy()
    )

texture_visuals_after = trimesh.visual.texture.TextureVisuals(
    uv=uv2[:len(mesh2_after.vertices)],
    material=material
)
mesh2_after.visual = texture_visuals_after

scene.add_geometry(mesh2_after)

scene.show()

In [42]:
SAMPLES = 17
ice = px.colors.sample_colorscale(
    
    # DT4D
    px.colors.cyclical.Edge,
    
    # FAUST
    # px.colors.sequential.Jet,
    
    # SHREC19
    # px.colors.diverging.Picnic,
    
    # SCAPE
    # px.colors.cyclical.HSV,
    
    # px.colors.cyclical.IceFire,
    
    
    # px.colors.sequential.Blackbody,
    # px.colors.sequential.Viridis,
    
    SAMPLES)

rgb = [px.colors.unconvert_from_RGB_255(px.colors.unlabel_rgb(c)) for c in ice]

cmap = mcolors.ListedColormap(rgb, name='Ice', N=SAMPLES)


In [43]:
cmap

In [40]:
rgb

# for every entry, multiply it by 255
# then convert to int

for i in range(SAMPLES):
    print(f'rgb({int(255*rgb[i][0])}, {int(255*rgb[i][1])}, {int(255*rgb[i][2])})')

In [38]:
print(cmap(1))